In [1]:
# import all modules

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Device config

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper Parameters

num_epochs = 40
batch_size = 64
learning_rate = 0.001


In [12]:
pip install -U imbalanced-learn


  Attempting uninstall: imbalanced-learn
    Found existing installation: imbalanced-learn 0.10.1
    Uninstalling imbalanced-learn-0.10.1:
      Successfully uninstalled imbalanced-learn-0.10.1


In [1]:
from imblearn.over_sampling import SMOTE

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, random_split

# Define your transforms
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to 32x32
    transforms.ToTensor(),        # Convert PIL Image to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to the same range as CIFAR-10
])

# Load your full dataset
full_dataset = datasets.ImageFolder(root="C:\\Users\\44778\\OneDrive\\Desktop\\UWE_Docs\\7. Dissertation\\CNN Data Files\\MFCC", transform=transform)

# Determine the sizes for the train and test splits
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size

# Split the dataset into training and test sets
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

# Create DataLoader for training and test sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

classes = ('bellbird','haast kiwi','kaka','kea','morepork','other kiwi birds','paradise shelduck','swamphen')

# Define the AlexNet class
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 8) -> None:  # Set num_classes to match the number of classes
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # Adjusted kernel size to 3x3 to preserve dimensions
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Pooling with kernel size 2x2, stride 2
            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Final pooling layer
        )

        # Adjusting avgpool to output (2, 2) size
        self.avgpool = nn.AdaptiveAvgPool2d((2, 2))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 2 * 2, 1024),  # Adjusted linear layer to match the avgpool output
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AlexNet(num_classes=len(classes)).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training the model
num_epochs = 10
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 200 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

# Ensure the model is in evaluation mode
model.eval()

# Initialize counters for accuracy calculation
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for _ in range(len(classes))]
    n_class_samples = [0 for _ in range(len(classes))]

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        
        # Get predictions
        _, predicted = torch.max(outputs, 1)
        
        # Update sample count
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        # Update class-specific counts
        for i in range(labels.size(0)):
            label = labels[i].item()
            pred = predicted[i].item()
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    # Calculate and print the overall accuracy
    overall_acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {overall_acc:.2f}%')
    
    # Calculate and print class-specific accuracies
    for i in range(len(classes)):
        if n_class_samples[i] > 0:
            class_acc = 100.0 * n_class_correct[i] / n_class_samples[i]
            print(f'Accuracy of {classes[i]}: {class_acc:.2f}%')
        else:
            print(f'No samples for class {classes[i]} in the test set.')


Finished Training
Accuracy of the network: 19.83%
Accuracy of bellbird: 0.00%
Accuracy of haast kiwi: 0.00%
Accuracy of kaka: 0.00%
Accuracy of kea: 0.00%
Accuracy of morepork: 0.00%
Accuracy of other kiwi birds: 100.00%
Accuracy of paradise shelduck: 0.00%
Accuracy of swamphen: 0.00%
